In [2]:
!pip install langchain_google_genai langchain_community langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
 

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from google.colab import userdata
# 1. Setup API Keys
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["TAVILY_API_KEY"] = "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"
 # Get one free at tavily.com


# 2. Define your local tool
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    #return f"It's always sunny in {city}!"

# 3. Initialize Search (Tavily is the 2025 standard for LLM search)
search_tool = TavilySearchResults(max_results=2)

# 4. Initialize Gemini
# gemini-2.0-flash is highly recommended for agents due to speed/accuracy
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# 5. Combine tools
tools = [get_weather, search_tool]

# 6. Create the Agent
# In 2025, create_react_agent is the unified way to handle the Re-Act loop
agent = create_react_agent(
    model=llm,
    tools=tools
    system_prompt="You have access to get_weather and search_tool."
)

# 7. Run the Agent
inputs = {"messages": [("user", "What is the weather in NYC and who is the current mayor?")]}

print("--- Agent is processing ---")
for chunk in agent.stream(inputs, stream_mode="values"):
    # This prints the progression of the conversation
    message = chunk["messages"][-1]
    if hasattr(message, "content") and message.content:
        print(f"\n[{message.type.upper()}]: {message.content}")

/tmp/ipython-input-1427921.py:20: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)
/tmp/ipython-input-1427921.py:31: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


--- Agent is processing ---

[HUMAN]: What is the weather in NYC and who is the current mayor?

[TOOL]: [{"title": "Mayor of New York City - Wikipedia", "url": "https://en.wikipedia.org/wiki/Mayor_of_New_York_City", "content": "The current mayor is Eric Adams, who was elected on November 2, 2021, and took office shortly after midnight on January 1, 2022. Zohran Mamdani is the mayor-elect, having won the election held on November 4, 2025. Mamdani will take office on January 1, 2026.\n\n## History [...] | Mayor of the City of New York | |\n --- |\n| Seal of the City of New York | |\n| Standard of the mayor of New York City | |\n| Incumbent Eric Adams since January 1, 2022 | |\n| Government of New York City | |\n| Style \"Style (form of address)\") | His Honor; Mr. Mayor (informal) |\n| Residence | Gracie Mansion |\n| Seat | New York City Hall |\n| Term length | Four years, renewable once consecutively |\n| Constituting instrument | New York City Charter |\n| Inaugural holder | Thomas Wil